# AdaBoost

Codigo fuente: https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html#sphx-glr-auto-examples-ensemble-plot-adaboost-regression-py

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import make_scorer, roc_auc_score

In [2]:
import pandas as pd
import os

In [3]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [4]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

In [5]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

ValueError: Wrong number of items passed 11, placement implies 1

In [6]:
rng = np.random.RandomState(1)

In [7]:
%run $set_de_entrenamiento_testing_y_prediccion

In [8]:
training = labels_training.merge(df_features, how='inner', on='person')

In [9]:
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)

regr_1.fit(training.drop('label', axis=1), training['label'])
regr_2.fit(training.drop('label', axis=1), training['label'])


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x1a22bf5b88>)

In [10]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [11]:
labels_test['label_predicted'] = regr_1.predict(labels_test.drop('label', axis=1))

In [12]:
%run $calcular_auc

auc score: 0.8229173939859424


In [13]:
labels_test['label_predicted'] = regr_2.predict(labels_test.drop(['label','label_predicted'], axis=1))

In [14]:
%run $calcular_auc

auc score: 0.8345945049574082


# Tuning an AdaBoost regressor

In [15]:
labels_with_features = labels.merge(df_features, how='inner', on='person')
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)

In [16]:
splits = 10 # cantidad de splits en el cross validation
n_iter_search = 20 # cantidad de combinaciones, en total splits*n_iter_search RF a probar

In [17]:
from sklearn.model_selection import RandomizedSearchCV


In [20]:
param_dist = {
 'n_estimators': [100,125,150,200],
 'learning_rate' : [0.01,0.05,0.1],
 'loss' : ['linear'],
    'base_estimator' :[DecisionTreeRegressor(max_depth=4),DecisionTreeRegressor(max_depth=6),DecisionTreeRegressor(max_depth=10)]
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=3,
 n_iter = 36,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x000001C37F010BD0>),
          fit_params=None, iid=True, n_iter=36, n_jobs=-1,
          param_distributions={'n_estimators': [100, 125, 150, 200], 'learning_rate': [0.01, 0.05, 0.1], 'loss': ['linear'], 'base_estimator': [DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, ...lit=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, s

El mejor AdaBoost fue:

In [21]:
print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

score: 0.8724676386192912


{'n_estimators': 125,
 'loss': 'linear',
 'learning_rate': 0.05,
 'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')}

# Seguir mejorando

In [24]:
param_dist = {
 'n_estimators': [124,125,126,127,128],
 'learning_rate' : [0.05,0.07,0.08],
 'loss' : ['linear'],
    'base_estimator' :[DecisionTreeRegressor(max_depth=3),DecisionTreeRegressor(max_depth=4),DecisionTreeRegressor(max_depth=5)]
    
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=3,
 n_iter = 45,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x000001C37F010BD0>),
          fit_params=None, iid=True, n_iter=45, n_jobs=-1,
          param_distributions={'n_estimators': [124, 125, 126, 127, 128], 'learning_rate': [0.05, 0.07, 0.08], 'loss': ['linear'], 'base_estimator': [DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=...lit=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, s

In [25]:
print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

score: 0.8744386593762937


{'n_estimators': 124,
 'loss': 'linear',
 'learning_rate': 0.07,
 'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')}

In [26]:
param_dist = {
 'n_estimators': [125,126,127,128,129],
 'learning_rate' : [0.04,0.05,0.06,0.07,0.08],
 'loss' : ['linear']
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=3,
 n_iter = 20,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x000001C37F010BD0>),
          fit_params=None, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': [125, 126, 127, 128, 129], 'learning_rate': [0.04, 0.05, 0.06, 0.07, 0.08], 'loss': ['linear']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=make_scorer(roc_auc_score),
          verbose=0)

In [27]:
print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

score: 0.8744386593762937


{'n_estimators': 128, 'loss': 'linear', 'learning_rate': 0.07}

In [23]:
param_dist = {
 'n_estimators': [128,129,130,131,132],
 'learning_rate' : [0.075,0.08,0.085,0.09,0.095],
 'loss' : ['linear']
 }

pre_gs_inst = RandomizedSearchCV(regr_2,
 param_distributions = param_dist,
 cv=3,
 n_iter = 20,
 n_jobs=-1,scoring=make_scorer(roc_auc_score))

pre_gs_inst.fit(training.drop('label', axis=1), training['label'])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=<mtrand.RandomState object at 0x00000214B7B013F0>),
          fit_params=None, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': [128, 129, 130, 131, 132], 'learning_rate': [0.075, 0.08, 0.085, 0.09, 0.095], 'loss': ['linear']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=make_scorer(roc_auc_score),
          verbose=0)

In [24]:
print('score: {}'.format(pre_gs_inst.best_score_))
pre_gs_inst.best_params_

score: 0.8473178056427262


{'n_estimators': 131, 'loss': 'linear', 'learning_rate': 0.095}

## Predecir labels desconocidos

Vamos a predecir los nuevos labels.

In [13]:
from sklearn.model_selection import cross_val_score

In [10]:
training = labels.merge(df_features, how='inner', on='person')

In [11]:
param = {
    'n_estimators': 124,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
}

In [12]:
regr_2 = AdaBoostRegressor(**param)

In [18]:
scores = cross_val_score(regr_2, training.drop('label', axis=1), training['label'], cv=10, scoring='roc_auc')

In [19]:
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.870942 (+/- 0.027855)


In [20]:
labels_to_predict_with_features = labels_to_predict.merge(df_features, how='inner', on='person')

In [23]:
assert(labels_to_predict.shape[0] == labels_to_predict_with_features.shape[0])

In [25]:
regr_2.fit(training.drop('label', axis=1), training['label'])

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=0.07, loss='linear', n_estimators=124,
         random_state=None)

In [26]:
labels_to_predict['label'] = regr_2.predict(labels_to_predict_with_features)

In [30]:
# labels_to_predict.to_csv(predicciones_csv)